# Build Docker image 

In this notebook, we will build the docker container that contains the ResNet152 model, Flask web application, model driver and all dependencies.
Make sure you have logged in using docker login.

In [1]:
import os
from os import path
import json
import shutil

In [2]:
os.makedirs('flaskwebapp', exist_ok=True)
os.makedirs(os.path.join('flaskwebapp', 'nginx'), exist_ok=True)
os.makedirs(os.path.join('flaskwebapp', 'etc'), exist_ok=True)

In [3]:
shutil.copy('resnet152.py', 'flaskwebapp')
shutil.copy('driver.py', 'flaskwebapp')
os.listdir('flaskwebapp')

['resnet152.py', 'etc', 'driver.py', 'nginx']

Below, we create the module for the Flask web application.

In [4]:
%%writefile flaskwebapp/app.py

from flask import Flask, request
import logging
import json
import driver

app = Flask(__name__)
predict_for = driver.get_model_api()
 
@app.route("/score", methods = ['POST'])
def scoreRRS():
    """ Endpoint for scoring
    """
    if request.headers['Content-Type'] != 'application/json':
        return Response(json.dumps({}), status= 415, mimetype ='application/json')
    request_input = request.json['input']
    response = predict_for(request_input)
    print(response)
    return json.dumps({'result': str(response)})


@app.route("/")
def healthy():
    return "Healthy"

# Tensorflow Version
@app.route('/version', methods = ['GET'])
def version_request():
    return driver.version()

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5000)

Writing flaskwebapp/app.py


In [5]:
%%writefile flaskwebapp/wsgi.py
from app import app as application

def create():
    print("Initialising")
    application.run(host='127.0.0.1', port=5000)

Writing flaskwebapp/wsgi.py


Here, we write the configuration for the Nginx which creates a proxy between ports **80** and **5000**.

In [6]:
%%writefile flaskwebapp/nginx/app
server {
    listen 80;
    server_name _;
 
    location / {
    include proxy_params;
    proxy_pass http://127.0.0.1:5000;
    proxy_connect_timeout 5000s;
    proxy_read_timeout 5000s;
  }
}

Writing flaskwebapp/nginx/app


In [7]:
%%writefile flaskwebapp/gunicorn_logging.conf

[loggers]
keys=root, gunicorn.error

[handlers]
keys=console

[formatters]
keys=json

[logger_root]
level=INFO
handlers=console

[logger_gunicorn.error]
level=ERROR
handlers=console
propagate=0
qualname=gunicorn.error

[handler_console]
class=StreamHandler
formatter=json
args=(sys.stdout, )

[formatter_json]
class=jsonlogging.JSONFormatter

Writing flaskwebapp/gunicorn_logging.conf


In [8]:
%%writefile flaskwebapp/kill_supervisor.py
import sys
import os
import signal

def write_stdout(s):
    sys.stdout.write(s)
    sys.stdout.flush()

# this function is modified from the code and knowledge found here: http://supervisord.org/events.html#example-event-listener-implementation
def main():
    while 1:
        write_stdout('READY\n')
        # wait for the event on stdin that supervisord will send
        line = sys.stdin.readline()
        write_stdout('Killing supervisor with this event: ' + line);
        try:
            # supervisord writes its pid to its file from which we read it here, see supervisord.conf
            pidfile = open('/tmp/supervisord.pid','r')
            pid = int(pidfile.readline());
            os.kill(pid, signal.SIGQUIT)
        except Exception as e:
            write_stdout('Could not kill supervisor: ' + e.strerror + '\n')
            write_stdout('RESULT 2\nOK')

main()

Writing flaskwebapp/kill_supervisor.py


In [9]:
%%writefile flaskwebapp/etc/supervisord.conf 
[supervisord]
logfile=/tmp/supervisord.log ; (main log file;default $CWD/supervisord.log)
logfile_maxbytes=50MB        ; (max main logfile bytes b4 rotation;default 50MB)
logfile_backups=10           ; (num of main logfile rotation backups;default 10)
loglevel=info                ; (log level;default info; others: debug,warn,trace)
pidfile=/tmp/supervisord.pid ; (supervisord pidfile;default supervisord.pid)
nodaemon=true                ; (start in foreground if true;default false)
minfds=1024                  ; (min. avail startup file descriptors;default 1024)
minprocs=200                 ; (min. avail process descriptors;default 200)

[program:gunicorn]
command=bash -c "gunicorn --workers 1 -m 007 --timeout 100000 --capture-output --error-logfile - --log-level debug --log-config gunicorn_logging.conf \"wsgi:create()\""
directory=/code
redirect_stderr=true
stdout_logfile =/dev/stdout
stdout_logfile_maxbytes=0
startretries=2
startsecs=20

[program:nginx]
command=/usr/sbin/nginx -g "daemon off;"
startretries=2
startsecs=5
priority=3

[eventlistener:program_exit]
command=python kill_supervisor.py
directory=/code
events=PROCESS_STATE_FATAL
priority=2

Writing flaskwebapp/etc/supervisord.conf


We create a custom image based on the CUDA 8 image from NVIDIA and install all the necessary dependencies. This is in order to try and keep the size of the image as small as possible.

In [10]:
%%writefile flaskwebapp/requirements.txt

Pillow==5.0.0
click==6.7
configparser==3.5.0
Flask==0.12.2
gunicorn==19.6.0
json-logging-py==0.2
MarkupSafe==1.0
olefile==0.44
requests==2.18.4
h5py==2.6.0

Writing flaskwebapp/requirements.txt


In [13]:
%%writefile flaskwebapp/dockerfile

FROM nvidia/cuda:9.0-cudnn7-devel-ubuntu16.04
    
RUN echo "deb http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64 /" > /etc/apt/sources.list.d/nvidia-ml.list

USER root
RUN mkdir /code
WORKDIR /code
RUN chmod -R a+w /code
ADD . /code/

RUN apt-get update && apt-get install -y --no-install-recommends \
        build-essential \
        ca-certificates \
        cmake \
        wget \
        curl \
        git \
        supervisor \
        nginx && \
    rm -rf /var/lib/apt/lists/* &&\
    rm /etc/nginx/sites-enabled/default && \
    cp /code/nginx/app /etc/nginx/sites-available/ && \
    ln -s /etc/nginx/sites-available/app /etc/nginx/sites-enabled/ 

ENV PYTHON_VERSION=3.5
RUN curl -o ~/miniconda.sh -O  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh  && \
    chmod +x ~/miniconda.sh && \
    ~/miniconda.sh -b -p /opt/conda && \
    rm ~/miniconda.sh && \
    /opt/conda/bin/conda create -y --name py$PYTHON_VERSION python=$PYTHON_VERSION numpy pyyaml scipy \
    ipython pandas jupyter ipykernel scikit-learn && \
    /opt/conda/bin/conda clean -ya
    
ENV PATH /opt/conda/envs/py$PYTHON_VERSION/bin:$PATH
ENV LD_LIBRARY_PATH /opt/conda/envs/py$PYTHON_VERSION/lib:/usr/local/cuda/lib64/:$LD_LIBRARY_PATH
    
RUN pip install --upgrade pip && \
    pip install tensorflow-gpu==1.9.0 && \
    pip install keras==2.2.0 && \
    pip install -r /code/requirements.txt && \       
    /opt/conda/bin/conda clean -yt

EXPOSE 8888
EXPOSE 5000
EXPOSE 80

CMD ["supervisord", "-c", "/code/etc/supervisord.conf"]

Overwriting flaskwebapp/dockerfile


The image name below refers to our dockerhub account. If you wish to push the image to your account make sure you change the docker login.

In [14]:
docker_login = 'fboylu'
image_name = docker_login + '/kerastf-gpu'
application_path = 'flaskwebapp'
docker_file_location = path.join(application_path, 'dockerfile')

In [15]:
!docker build -t $image_name -f $docker_file_location $application_path --no-cache

Sending build context to Docker daemon  33.79kB
Step 1/17 : FROM nvidia/cuda:9.0-cudnn7-devel-ubuntu16.04
 ---> 22d678ae96f5
Step 2/17 : RUN echo "deb http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64 /" > /etc/apt/sources.list.d/nvidia-ml.list
 ---> Running in 6f9000eb7c79
Removing intermediate container 6f9000eb7c79
 ---> 3da1b7dfb480
Step 3/17 : USER root
 ---> Running in d0c0ffeb2551
Removing intermediate container d0c0ffeb2551
 ---> f5ec105d3506
Step 4/17 : RUN mkdir /code
 ---> Running in 0b871b230194
Removing intermediate container 0b871b230194
 ---> 95aab01f5e8f
Step 5/17 : WORKDIR /code
Removing intermediate container afa810f3dbf0
 ---> be6af228c42d
Step 6/17 : RUN chmod -R a+w /code
 ---> Running in 35d925c42fd4
Removing intermediate container 35d925c42fd4
 ---> 3dff918dc6ec
Step 7/17 : ADD . /code/
 ---> 5fc3247d047a
Step 8/17 : RUN apt-get update && apt-get install -y --no-install-recommends         build-essential         ca-certificates

Get:25 http://archive.ubuntu.com/ubuntu xenial/main amd64 libxcb1 amd64 1.11.1-1ubuntu1 [40.0 kB]
Get:26 http://archive.ubuntu.com/ubuntu xenial/main amd64 libx11-data all 2:1.6.3-1ubuntu2 [113 kB]
Get:27 http://archive.ubuntu.com/ubuntu xenial/main amd64 libx11-6 amd64 2:1.6.3-1ubuntu2 [571 kB]
Get:28 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 wget amd64 1.17.1-1ubuntu1.4 [299 kB]
Get:29 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 curl amd64 7.47.0-1ubuntu2.8 [139 kB]
Get:30 http://archive.ubuntu.com/ubuntu xenial/main amd64 fonts-dejavu-core all 2.35-1 [1039 kB]
Get:31 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 fontconfig-config all 2.11.94-0ubuntu1.1 [49.9 kB]
Get:32 http://archive.ubuntu.com/ubuntu xenial/main amd64 liberror-perl all 0.17-1.2 [19.6 kB]
Get:33 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 git-man all 1:2.7.4-0ubuntu1.4 [736 kB]
Get:34 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 git amd64 1:2.

Selecting previously unselected package liberror-perl.
Preparing to unpack .../liberror-perl_0.17-1.2_all.deb ...
Unpacking liberror-perl (0.17-1.2) ...
Selecting previously unselected package git-man.
Preparing to unpack .../git-man_1%3a2.7.4-0ubuntu1.4_all.deb ...
Unpacking git-man (1:2.7.4-0ubuntu1.4) ...
Selecting previously unselected package git.
Preparing to unpack .../git_1%3a2.7.4-0ubuntu1.4_amd64.deb ...
Unpacking git (1:2.7.4-0ubuntu1.4) ...
Selecting previously unselected package libfreetype6:amd64.
Preparing to unpack .../libfreetype6_2.6.1-0.1ubuntu2.3_amd64.deb ...
Unpacking libfreetype6:amd64 (2.6.1-0.1ubuntu2.3) ...
Selecting previously unselected package libfontconfig1:amd64.
Preparing to unpack .../libfontconfig1_2.11.94-0ubuntu1.1_amd64.deb ...
Unpacking libfontconfig1:amd64 (2.11.94-0ubuntu1.1) ...
Selecting previously unselected package libjpeg8:amd64.
Preparing to unpack .../libjpeg8_8c-2ubuntu8_amd64.deb ...
Unpacking libjpeg8:amd64 (8c-2ubuntu8) ...
Selecting p

Solving environment: ...working... done

## Package Plan ##

  environment location: /opt/conda/envs/py3.5

  added / updated specs: 
    - ipykernel
    - ipython
    - jupyter
    - numpy
    - pandas
    - python=3.5
    - pyyaml
    - scikit-learn
    - scipy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    tornado-5.0.2              |           py35_0         644 KB
    jedi-0.12.1                |           py35_0         227 KB
    libxml2-2.9.8              |       h26e45fe_1         2.0 MB
    pyzmq-17.0.0               |   py35h14c3975_0         457 KB
    pcre-8.42                  |       h439df22_0         251 KB
    scikit-learn-0.19.1        |   py35hbf1f462_0         5.2 MB
    jupyter_console-5.2.0      |   py35h4044a63_1          35 KB
    pytz-2018.5                |           py35_0         231 KB
    sip-4.19.8                 |   py35hf484d3e_0         291 KB
    s

tornado-5.0.2        |  644 KB | ########## | 100% 
jedi-0.12.1          |  227 KB | ########## | 100% 
libxml2-2.9.8        |  2.0 MB | ########## | 100% 
pyzmq-17.0.0         |  457 KB | ########## | 100% 
pcre-8.42            |  251 KB | ########## | 100% 
scikit-learn-0.19.1  |  5.2 MB | ########## | 100% 
jupyter_console-5.2. |   35 KB | ########## | 100% 
pytz-2018.5          |  231 KB | ########## | 100% 
sip-4.19.8           |  291 KB | ########## | 100% 
six-1.11.0           |   21 KB | ########## | 100% 
gmp-6.1.2            |  744 KB | ########## | 100% 
entrypoints-0.2.3    |    9 KB | ########## | 100% 
jinja2-2.10          |  182 KB | ########## | 100% 
pyopenssl-18.0.0     |   82 KB | ########## | 100% 
automat-0.7.0        |   52 KB | ########## | 100% 
attrs-18.1.0         |   44 KB | ########## | 100% 
appdirs-1.4.3        |   16 KB | ########## | 100% 
glib-2.56.1          |  5.0 MB | ########## | 100% 
backcall-0.1.0       |   19 KB | ########## | 100% 
pyqt-5.9.2  

Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done

#
# To activate this environment, use:
# > source activate py3.5
#
# To deactivate an active environment, use:
# > source deactivate
#

Cache location: /opt/conda/pkgs
Will remove the following tarballs:

/opt/conda/pkgs
---------------
openssl-1.0.2o-h20670df_0.tar.bz2            3.4 MB
attrs-18.1.0-py35_0.tar.bz2                   44 KB
certifi-2018.4.16-py35_0.tar.bz2             143 KB
automat-0.7.0-py35_0.tar.bz2                  52 KB
ruamel_yaml-0.15.37-py36h14c3975_2.tar.bz2     245 KB
cffi-1.11.5-py36h9745a5d_0.tar.bz2           212 KB
sqlite-3.24.0-h84994c4_0.tar.bz2             1.8 MB
readline-7.0-ha6073c6_4.tar.bz2              1.1 MB
asn1crypto-0.24.0-py36_0.tar.bz2             155 KB
pandocfilters-1.4.2-py35h1565a15_1.tar.bz2      12 KB
wcwidth-0.1.7-py35hcd08066_0.tar.bz2          25 KB
jsonschema-2.6.0-py35h4395190_0.tar.bz2       63 KB
parso-0.

Removed pyasn1-modules-0.2.2-py35_0.tar.bz2
Removed jinja2-2.10-py35h480ab6d_0.tar.bz2
Removed jupyter-1.0.0-py35_4.tar.bz2
Removed tk-8.6.7-hc745277_3.tar.bz2
Removed libxcb-1.13-h1bed415_1.tar.bz2
Removed pandoc-2.2.1-h629c226_0.tar.bz2
Removed nbformat-4.4.0-py35h12e6e07_0.tar.bz2
Removed pysocks-1.6.8-py36_0.tar.bz2
Removed mkl_fft-1.0.4-py35h4414c95_1.tar.bz2
Removed jupyter_console-5.2.0-py35h4044a63_1.tar.bz2
Removed prompt_toolkit-1.0.15-py35hc09de7a_0.tar.bz2
Removed html5lib-1.0.1-py35h2f9c1c0_0.tar.bz2
Removed bleach-2.1.3-py35_0.tar.bz2
Removed terminado-0.8.1-py35_1.tar.bz2
Removed idna-2.7-py35_0.tar.bz2
Removed setuptools-39.2.0-py35_0.tar.bz2
Removed notebook-5.6.0-py35_0.tar.bz2
Removed pyyaml-3.13-py35h14c3975_0.tar.bz2
Removed pycparser-2.18-py35h61b3040_1.tar.bz2
Removed pickleshare-0.7.4-py35hd57304d_0.tar.bz2
Removed pcre-8.42-h439df22_0.tar.bz2
Removed libedit-3.1.20170329-h6b74fdf_2.tar.bz2
Removed six-1.11.0-py35h423b573_1.tar.bz2
Removed requests-2.18.4-py36he

  Running setup.py bdist_wheel for termcolor: started
  Running setup.py bdist_wheel for termcolor: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/7c/06/54/bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6
  Running setup.py bdist_wheel for gast: started
  Running setup.py bdist_wheel for gast: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/9a/1f/0e/3cde98113222b853e98fc0a8e9924480a3e25f1b4008cedb4f
  Running setup.py bdist_wheel for absl-py: started
  Running setup.py bdist_wheel for absl-py: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/4c/16/ef/e36a23f2432e9220f8845f94e2c3abd39e7d9d1cd458d3159d
Successfully built termcolor gast absl-py
  Found existing installation: setuptools 39.2.0
    Uninstalling setuptools-39.2.0:
      Successfully uninstalled setuptools-39.2.0
  Running setup.py bdist_wheel for configparser: started


  Running setup.py bdist_wheel for configparser: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/a3/61/79/424ef897a2f3b14684a7de5d89e8600b460b89663e6ce9d17c
  Running setup.py bdist_wheel for json-logging-py: started
  Running setup.py bdist_wheel for json-logging-py: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/0d/2e/1c/c638b7589610d8b9358a6e5eb008edacb8b3e9b6d1edc9479f
  Running setup.py bdist_wheel for olefile: started
  Running setup.py bdist_wheel for olefile: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/c4/19/76/61fc7929d808e51567aff23036ca5fe6ba8336ad0559ca6a27
  Running setup.py bdist_wheel for itsdangerous: started
  Running setup.py bdist_wheel for itsdangerous: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/2c/4a/61/5599631c1554768c6290b08c02c72d7317910374ca602ff1e5
Successfully built configparser json-logging-py olefile itsdangerous
  Found existing installatio

In [16]:
!docker push $image_name

The push refers to repository [docker.io/fboylu/kerastf-gpu]

fe9fdae0: Preparing 
14727aa7: Preparing 
655e87b5: Preparing 
d93eef68: Preparing 
428999ff: Preparing 
13bbc74e: Preparing 
25fd50f4: Preparing 
586783ab: Preparing 
d5ec848e: Preparing 
015fd93f: Preparing 
f8205d06: Preparing 
c0dd0ec8: Preparing 
b5cb2442: Preparing 
91e51d73: Preparing 
d9e65295: Preparing 
45e78935: Preparing 
1dc646ba: Preparing 


4727aa7: Pushing  999.3MB/2.045GBsres50tf-gpu [16APushing  859.1MB/2.045GB

4727aa7: Pushed   2.121GB/2.045GBPushing  1.406GB/2.045GBlatest: digest: sha256:08f5f19a477b5e2953fdb6d74f5a2f826da0e39ddac78a6646c7955ea6fbf49a size: 4090


In [17]:
print('Docker image name {}'.format(image_name))

Docker image name fboylu/kerastf-gpu


We can now [test our image locally](03_TestLocally.ipynb).